# PNG 형식을 pickle 파일로 바꿔주는 코드

## 1) 데이터 생성을 위한 함수

In [1]:
from __future__ import print_function
from __future__ import absolute_import

import argparse
import glob
import os
import pickle as pickle
import random

In [2]:
### 이 함수는 각각의 학습이미지를 열어서 검증데이터와 학습데이터로 파일을 나눠주는 코드이다.
### from_dir : 학습시킬 데이터의 이미지 경로
### train_path : 학습데이터 저장경로
### val_path : 검증데이터 저장경로
### train_val_split : 검증데이터 비율
### with_charid = 뭔지 모르겠으나 True일 경우 label, charid, img_bytes 값을 label, charid 모든경우에 저장
###                               False일 경우 label, img_bytes 값을 label 경우에만 저장

def pickle_examples(from_dir, train_path, val_path, train_val_split=0.2, with_charid=False):
    """
    Compile a list of examples into pickled format, so during
    the training, all io will happen in memory
    """
    
    paths = glob.glob(os.path.join(from_dir, "*.png"))         ## from_dir 경로에 존재하는 모든 png 파일 입력
    with open(train_path, 'wb') as ft:                        ## with as 구문 파일을 자동으로 열고 닫아주는 구문
        with open(val_path, 'wb') as fv:
            print('all data num:', len(paths))
            c = 1
            val_count = 0
            train_count = 0
            if with_charid:
                print('pickle with charid')
                for p in paths:
                    c += 1
                    label = int(os.path.basename(p).split("_")[0])  ## 해당 파일을 _ 단위로 나눠서 앞에것을 int형식 저장
                    charid = int(os.path.basename(p).split("_")[1].split(".")[0])     ## 뒤에것을 확장자 제거하고 저장
                    with open(p, 'rb') as f:
                        img_bytes = f.read()
                        example = (label, charid, img_bytes)
                        r = random.random()
                        if r < train_val_split:
                            pickle.dump(example, fv)
                            val_count += 1
                            if val_count % 10000 == 0:
                                print("%d imgs saved in val.obj" % val_count)
                        else:
                            pickle.dump(example, ft)
                            train_count += 1
                            if train_count % 10000 == 0:
                                print("%d imgs saved in train.obj" % train_count)
                print("%d imgs saved in val.obj, end" % val_count)
                print("%d imgs saved in train.obj, end" % train_count)
            else:
                for p in paths:
                    c += 1
                    label = int(os.path.basename(p).split("_")[0])
                    with open(p, 'rb') as f:
                        img_bytes = f.read()
                        example = (label, img_bytes)
                        r = random.random()
                        if r < train_val_split:
                            pickle.dump(example, fv)
                            val_count += 1
                            if val_count % 10000 == 0:
                                print("%d imgs saved in val.obj" % val_count)
                        else:
                            pickle.dump(example, ft)
                            train_count += 1
                            if train_count % 10000 == 0:
                                print("%d imgs saved in train.obj" % train_count)
                print("%d imgs saved in val.obj, end" % val_count)
                print("%d imgs saved in train.obj, end" % train_count)
            return

In [3]:
### 이 함수는 입력한 char_ids와 font_filter와 동일한 데이터를 찾는 경우 save_path에 데이터를 저장해주는 함수이다
### from_dir : 학습시킬 데이터의 이미지 경로
### save_path : 일치하는 데이터를 저장시킬 경로
### char_ids : 데이터가 일치하는지 확인하기 위한 데이터 번호
### font_filter : 데이터가 일치하는지 확인하기 위한 데이터 필터

def pickle_interpolation_data(from_dir, save_path, char_ids, font_filter):
    paths = glob.glob(os.path.join(from_dir, "*.png"))         ## from_dir 경로에 존재하는 모든 png 파일 입력
    with open(save_path, 'wb') as ft:
        c = 0
        for p in paths:
            charid = int(p.split('/')[-1].split('.')[0].split('_')[1]) ## 해당 파일을 _ 단위로 나눠서 뒤에것을 int형식 저장
            label = int(os.path.basename(p).split("_")[0])             ## 앞에것을 확장자 제거하고 저장
            if (charid in char_ids) and (label in font_filter): ## char_ids, font_filter에 해당 파일이름이 존재하면 작업수행
                c += 1
                with open(p, 'rb') as f:
                    img_bytes = f.read()
                    example = (label, charid, img_bytes)
                    pickle.dump(example, ft)
        print('data num:', c)
        return

## 2) 데이터 생성 코드

In [28]:
from_dir = '../get_data/dataset-11172/'
train_path = '../get_data/dataset_pkl/train.pickle'
val_path = '../get_data/dataset_pkl/val.pickle'
save_path = '../get_data/dataset_pkl/data.pickle'

In [29]:
pickle_examples(from_dir, train_path, val_path, train_val_split=0.2, with_charid=False)

all data num: 58802
10000 imgs saved in train.obj
20000 imgs saved in train.obj
30000 imgs saved in train.obj
10000 imgs saved in val.obj
40000 imgs saved in train.obj
11856 imgs saved in val.obj, end
46946 imgs saved in train.obj, end


In [22]:
pickle_interpolation_data(from_dir, save_path, list(range(1,35)), list(range(1,3001)))

data num: 669
